# Gray - Scott Model of Reaction - Diffusion

A Reaction-Diffusion Model is a mathematical model which generally describes the interdependent interaction between objects in time and space. They are often used in chemistry to simulate the interaction between various chemical interactions in a given space. The Gray - Scott Model is one such reaction diffusion model.

The system in continuous form is described by the system of differential equations,
$$ \frac{\partial u}{\partial t} = D_u \nabla^2 u - uv^2 + f(1-u)$$
$$ \frac{\partial v}{\partial t} = D_v \nabla^2 v - uv^2 - (f+k)v$$

where $u, v$ are the associated substance spaces in euclidian space reacting with one another, $D_u$ is the diffusion coefficient of $u$, $D_v$ is the diffusion coefficient of $v$, $f$ is the feed rate of $u$, $k$ is the kill rate of $v$. $\nabla^2$ describes the laplacian of $u$ and $v$.

In the discretized form this looks like, 
$$ A' = A + (D_A \nabla^2 A - AB^2 + f(1-a))\Delta t $$
$$ B' = B + (D_B \nabla^2 B - AB^2 - (k+f)B)\Delta t $$

where $A$ and $B$ are the two substances in discretized space, $\Delta t$ is the discrete time step interval, and $\nabla^2$ describes the laplacian of $A$ and $B$. 

You can think of $\nabla^2$ being the $f$ in $f(x)$ but in this case, it would be $\nabla^2(x)$. Be careful here, because this isn't the gradient function $\nabla(x)$ squared, $\nabla^2$ is notation to describe the laplace operator, which is a function used to describe the divergence of the gradient of a scalar function in Euclidian space.




In [ ]:
# Packages Used: gray_scott.jl

## Laplacian 

In [ ]:
function laplacian(u)
    # Second Order Finite Difference 
    laplacian = zeros(10,10);

    for i in 2:11, j in 2:11
        laplacian[i-1,j-1] = u[i+1,j] + u[i-1,j] + u[i,j+1] + u[i,j-1] - 4*u[i,j]
    end

    return laplacian;
end

## Boundary Conditions

In [ ]:
function periodic_bc(u)
    # Periodic Boundary Conditions
    u[1,:] = u[end-1,:]
    u[end,:] = u[2,:]
    u[:,1] = u[:,end-1]
    u[:,end] = u[:,2]
end

## Gray-Scott Algorithm

In [ ]:
function gray_scott(U, V, Du, Dv, f, k)
    #=
    U - 'Chemical A' array 
    V - 'Chemical B' array
    Du - Diffusion Rate of A
    Dv - Diffusion Rate of B
    f - Feed Rate of A
    k - Kill Rate of B

    The formulas are given as follows:

    ∂u/∂t = Du∇²u - uv² + f(1-u)
    ∂v/∂t = Dv∇²v + uv² - (f+k)v

    where ∇² is the Laplacian operator, numerically approximated using 
    the second finite difference.
    =# 

    u, v = U[2:end-1, 2:end-1], V[2:end-1, 2:end-1];
    print("(u, v): Import of U and V || ")
    log(u, v);

    Lu = laplacian(U);
    Lv = laplacian(V);

    print("(Lu, Lv): Laplacian of U and V || ")
    log(Lu, Lv);

    uv² = u.*v.^2;
    u += Du*Lu - uv² .+ f*(1-u);
    v += Dv*Lv + uv² .- (f+k)*v;

    print("(u, v): After operations are applied to u and v || ")
    log(u, v);
    
    print('\n')
    print("(U, V): After operations are applied to U and V || ")
    log(U, V);

    periodic_bc(U);
    periodic_bc(V);

    print("(U, V): After periodic_bc is applied to U and V || ")
    log(U, V);

    return U, V;
end

## Initalization

## Simulation and Visualization